In [1]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
import numpy as np
import scipy.io as sio
from numpy import expand_dims
from numpy import asarray
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from skimage.filters import gabor_kernel
from scipy import ndimage as nd        
import matplotlib
import matplotlib.pyplot as plt    
from numpy.linalg import norm
from keras.optimizers import SGD, Adam, Adagrad
import graphviz
import pydot

# Define the Path and prepare the Dataset

In [2]:
data_path = 'E:/Sushree/Dataset/'

img_path = os.path.join(data_path,'Animals_with_Attributes2/JPEGImages/')
print(img_path)

print(len(os.listdir(img_path)))

def get_imlist(path, option):
    if option == 'jpg':
        return[os.path.join(path, f) for f in os.listdir(path) if f.endswith('.jpg')]
    
def prepare_dataset(img_path, width, height, option):
    folder_list = os.listdir(img_path)
    num_classes = len(folder_list)
    images = []
    labels = []
    for i in range(len(folder_list)):
        print(i)
        img_list = get_imlist(os.path.join(img_path, folder_list[i]), option)
        for j in range(len(img_list)):
            img = image.load_img(img_list[j], target_size = (width, height))
            img = np.array(img)
            #img = np.true_divide(img,[255.0], out = None)
            img = np.divide(np.subtract(img, np.mean(img)), np.std(img))
            x = image.img_to_array(img)
            images.append(x)
            labels.append(i)
    images = np.array(images, dtype = np.float32)                             
    labels = to_categorical(labels, num_classes)                             
    return images, labels  


width, height, ch = 224, 224, 3
images, labels = prepare_dataset(img_path, width, height,'jpg')

print(images.shape)
print(len(labels))

E:/Sushree/Dataset/Animals_with_Attributes2/JPEGImages/
50
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
(37322, 224, 224, 3)
37322


# Prepare data for Training

In [3]:

split_path = os.path.join(data_path,'data/xlsa17/data/AWA2/att_splits.mat')
matcontent = sio.loadmat(split_path)
trainval_loc = matcontent['trainval_loc'].squeeze() - 1

#print(matcontent)

print(trainval_loc, len(trainval_loc))

train_images = images[trainval_loc]
train_labels = labels[trainval_loc]
print(train_images.shape)
print(len(train_labels))


[21482 11452 21448 ...  7992 10388  5160] 23527
(23527, 224, 224, 3)
23527


# Define the CNN as a Feature Extractor

In [4]:
model = tf.keras.applications.resnet.ResNet101(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000
)

model.summary()

x = model.output
out = GlobalAveragePooling2D()(x)

new_model = Model(inputs = model.input, outputs = out)
new_model.summary()


for layer in new_model.layers:
    layer.trainable = False
    

Model: "resnet101"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                       

                                                                                                  
 conv2_block2_add (Add)         (None, None, None,   0           ['conv2_block1_out[0][0]',       
                                256)                              'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, None, None,   0           ['conv2_block2_add[0][0]']       
                                256)                                                              
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, None, None,   16448       ['conv2_block2_out[0][0]']       
                                64)                                                               
                                                                                                  
 conv2_blo

                                128)                                                              
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, None, None,   512        ['conv3_block2_2_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, None, None,   0          ['conv3_block2_2_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, None, None,   66048       ['conv3_block2_2_relu[0][0]']    
                                512)                                                              
          

 conv4_block1_1_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_1_conv[0][0]']    
 ization)                       256)                                                              
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, None, None,   0          ['conv4_block1_1_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block1_2_conv (Conv2D)   (None, None, None,   590080      ['conv4_block1_1_relu[0][0]']    
                                256)                                                              
                                                                                                  
 conv4_block1_2_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_2_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block3_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block3_3_conv[0][0]']    
 ization)                       1024)                                                             
                                                                                                  
 conv4_block3_add (Add)         (None, None, None,   0           ['conv4_block2_out[0][0]',       
                                1024)                             'conv4_block3_3_bn[0][0]']      
                                                                                                  
 conv4_block3_out (Activation)  (None, None, None,   0           ['conv4_block3_add[0][0]']       
                                1024)                                                             
                                                                                                  
 conv4_blo

...

 conv4_block16_1_conv (Conv2D)  (None, None, None,   262400      ['conv4_block15_out[0][0]']      
                                256)                                                              
                                                                                                  
 conv4_block16_1_bn (BatchNorma  (None, None, None,   1024       ['conv4_block16_1_conv[0][0]']   
 lization)                      256)                                                              
                                                                                                  
 conv4_block16_1_relu (Activati  (None, None, None,   0          ['conv4_block16_1_bn[0][0]']     
 on)                            256)                                                              
                                                                                                  
 conv4_block16_2_conv (Conv2D)  (None, None, None,   590080      ['conv4_block16_1_relu[0][0]']   
          

                                                                                                  
 conv4_block18_add (Add)        (None, None, None,   0           ['conv4_block17_out[0][0]',      
                                1024)                             'conv4_block18_3_bn[0][0]']     
                                                                                                  
 conv4_block18_out (Activation)  (None, None, None,   0          ['conv4_block18_add[0][0]']      
                                1024)                                                             
                                                                                                  
 conv4_block19_1_conv (Conv2D)  (None, None, None,   262400      ['conv4_block18_out[0][0]']      
                                256)                                                              
                                                                                                  
 conv4_blo

 on)                            256)                                                              
                                                                                                  
 conv4_block21_3_conv (Conv2D)  (None, None, None,   263168      ['conv4_block21_2_relu[0][0]']   
                                1024)                                                             
                                                                                                  
 conv4_block21_3_bn (BatchNorma  (None, None, None,   4096       ['conv4_block21_3_conv[0][0]']   
 lization)                      1024)                                                             
                                                                                                  
 conv4_block21_add (Add)        (None, None, None,   0           ['conv4_block20_out[0][0]',      
                                1024)                             'conv4_block21_3_bn[0][0]']     
          

 conv5_block1_2_conv (Conv2D)   (None, None, None,   2359808     ['conv5_block1_1_relu[0][0]']    
                                512)                                                              
                                                                                                  
 conv5_block1_2_bn (BatchNormal  (None, None, None,   2048       ['conv5_block1_2_conv[0][0]']    
 ization)                       512)                                                              
                                                                                                  
 conv5_block1_2_relu (Activatio  (None, None, None,   0          ['conv5_block1_2_bn[0][0]']      
 n)                             512)                                                              
                                                                                                  
 conv5_block1_0_conv (Conv2D)   (None, None, None,   2099200     ['conv4_block23_out[0][0]']      
          

                                                                                                  
 conv5_block3_out (Activation)  (None, None, None,   0           ['conv5_block3_add[0][0]']       
                                2048)                                                             
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['conv5_block3_out[0][0]']       
 alAveragePooling2D)                                                                              
                                                                                                  
Total params: 42,658,176
Trainable params: 42,552,832
Non-trainable params: 105,344
__________________________________________________________________________________________________


# Extract Visual features for the training set

In [5]:
batch_size = 16

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y
    
trainval_gen = DataGenerator(train_images, train_labels, batch_size)  
    
train_visual_features = new_model.predict(trainval_gen, steps = len(train_images)//batch_size, verbose = 1)
print(train_visual_features.shape)

1470/1470 [==============================] - 62s 38ms/step
(23520, 2048)


In [6]:
print(np.max(train_visual_features))
print(np.min(train_visual_features))

21.353685
0.0


# Load transformed (Kernel PCA) gabor features for the train set

In [7]:

train_gabor_feature_transformed = np.load('C:/Users/Admin/Sushree_Codes/Sush/Results/gabor_feature_8_kernels_AWA2_train_tpoly_2048.npy')
print(train_gabor_feature_transformed, train_gabor_feature_transformed.shape)

[[ 0.9689931   0.00987228 -0.1609496  ... -0.09204392  0.14142427
   0.07633436]
 [-0.15023941 -0.10419849  0.06646418 ... -0.00454554 -0.00204696
   0.00983908]
 [ 0.6123006   0.4701565  -0.1773438  ...  0.00241815 -0.01357905
  -0.02886768]
 ...
 [ 0.06685977 -0.3986422  -0.24804702 ... -0.00297031  0.00263564
   0.00414803]
 [-0.7090743   0.34373647 -0.41183546 ...  0.00482323 -0.0147576
   0.00113354]
 [-0.31808448 -0.6135152  -0.20063762 ... -0.00361166  0.01295767
   0.01796482]] (23527, 2048)


In [8]:
class FeatGenerator(Sequence):
    def __init__(self, x_set, batch_size):
        self.x = x_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x

num_features, feat_len = np.shape(train_gabor_feature_transformed)

feat = Input(shape = (feat_len))
flat = Flatten()(feat)

model_transform_handcrafted = Model(inputs = feat, outputs = flat)
model_transform_handcrafted.summary()

train_gabor_feature_gen = FeatGenerator(train_gabor_feature_transformed, batch_size)
transformed_handcrafted_features = model_transform_handcrafted.predict(train_gabor_feature_gen, steps = len(train_images)//batch_size, verbose = 1)

print(transformed_handcrafted_features, transformed_handcrafted_features.shape)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 2048)]            0         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________
1470/1470 [==============================] - 1s 550us/step
[[ 0.9689931   0.00987228 -0.1609496  ... -0.09204392  0.14142427
   0.07633436]
 [-0.15023941 -0.10419849  0.06646418 ... -0.00454554 -0.00204696
   0.00983908]
 [ 0.6123006   0.4701565  -0.1773438  ...  0.00241815 -0.01357905
  -0.02886768]
 ...
 [-0.33896342 -0.08772518  0.2458664  ...  0.0117101  -0.00305183
  -0.00112637]
 [-0.4218307   0.6524654   0.43744814 ... -0.03178237  0.01439573
   0.00

In [9]:
print(np.max(transformed_handcrafted_features))
print(np.min(transformed_handcrafted_features))

5.5955687
-2.3678148


# Perform visual to handcrafted feature embedding

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_visual_features = scaler.fit_transform(train_visual_features)
print(train_visual_features, train_visual_features.shape)
print(np.max(train_visual_features))
print(np.min(train_visual_features))


In [ ]:
from sklearn.manifold import TSNE
import pandas as pd  
import seaborn as sns

def prepare_labels(img_path, option):
    folder_list = os.listdir(img_path)
    num_classes = len(folder_list)
    labels = []
    for i in range(len(folder_list)):
        print(i)
        img_list = get_imlist(os.path.join(img_path, folder_list[i]), option)
        for j in range(len(img_list)):
            labels.append(i)
    return labels  

labels_no_categorical = prepare_labels(img_path, 'jpg')

print(len(labels_no_categorical))

train_labels_no_categorical = np.zeros(len(trainval_loc))
for i in range(len(trainval_loc)):
    train_labels_no_categorical[i] = labels_no_categorical[trainval_loc[i]]
    
print(len(train_labels_no_categorical))

num_train_classes = 40
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(train_visual_features) 

print(z.shape)

df = pd.DataFrame()
df["y"] = train_labels_no_categorical[0:23520]
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(), palette=sns.color_palette("hls", num_train_classes), data=df).set(title="T-SNE projection")


In [ ]:

scaler2 = MinMaxScaler(feature_range=(-1, 1))
transformed_handcrafted_features = scaler2.fit_transform(transformed_handcrafted_features)
print(transformed_handcrafted_features, transformed_handcrafted_features.shape)
print(np.max(transformed_handcrafted_features))
print(np.min(transformed_handcrafted_features))

In [ ]:
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(transformed_handcrafted_features) 

print(z.shape)

df = pd.DataFrame()
df["y"] = train_labels_no_categorical[0:23520]
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(), palette=sns.color_palette("hls", num_train_classes), data=df).set(title="T-SNE projection")


In [10]:
train_visual_features = np.expand_dims(train_visual_features, axis=1)
train_visual_features = np.expand_dims(train_visual_features, axis=1)
print(train_visual_features, train_visual_features.shape)

[[[[1.1698666  0.         0.         ... 0.00428112 0.0513923
    0.        ]]]


 [[[0.9588352  0.1184229  0.         ... 0.         0.04434677
    0.        ]]]


 [[[0.01668728 0.         0.         ... 0.         0.01182768
    0.        ]]]


 ...


 [[[0.38645387 0.         0.         ... 0.         0.13524522
    0.        ]]]


 [[[0.96686786 0.         0.13401374 ... 0.0408974  0.51609856
    0.        ]]]


 [[[0.12148976 0.0894948  0.         ... 0.01861808 0.
    0.        ]]]] (23520, 1, 1, 2048)


In [11]:
transformed_handcrafted_features = np.expand_dims(transformed_handcrafted_features, axis=1)
transformed_handcrafted_features = np.expand_dims(transformed_handcrafted_features, axis=1)
print(transformed_handcrafted_features, transformed_handcrafted_features.shape)

[[[[ 0.9689931   0.00987228 -0.1609496  ... -0.09204392  0.14142427
     0.07633436]]]


 [[[-0.15023941 -0.10419849  0.06646418 ... -0.00454554 -0.00204696
     0.00983908]]]


 [[[ 0.6123006   0.4701565  -0.1773438  ...  0.00241815 -0.01357905
    -0.02886768]]]


 ...


 [[[-0.33896342 -0.08772518  0.2458664  ...  0.0117101  -0.00305183
    -0.00112637]]]


 [[[-0.4218307   0.6524654   0.43744814 ... -0.03178237  0.01439573
     0.00417776]]]


 [[[-0.02018051 -0.33419833  0.97277457 ...  0.01255293 -0.02157066
     0.01207785]]]] (23520, 1, 1, 2048)


In [12]:
num_features, feat_height, feat_width, feat_length = train_visual_features.shape

feat = Input(shape = (feat_height, feat_width, feat_length))

dense1 = Dense(1024, activation='tanh')(feat)
#conv1 = Conv2D(1024, 1, activation='relu')(feat)
#drop1 = Dropout(0.5)(dense1)

#dense2 = Dense(896, activation='relu')(dense1)
#drop2 = Dropout(0.5)(dense2)

dense3 = Dense(768, activation='tanh')(dense1)
#conv2 = Conv2D(768, 1,  activation='relu')(conv1)
#drop3 = Dropout(0.5)(dense3)

dense4 = Dense(512, activation='tanh')(dense3)
#conv3 = Conv2D(512, 1,  activation='relu')(conv2)
#drop4 = Dropout(0.5)(dense4)

dense5 = Dense(768, activation='tanh')(dense4)
#conv4 = Conv2D(768, 1,  activation='relu')(conv3)
#drop5 = Dropout(0.5)(dense5)

#dense6 = Dense(896, activation='relu')(dense5)
#drop6 = Dropout(0.5)(dense6)

dense7 = Dense(1024, activation='tanh')(dense5)
#conv5 = Conv2D(1024, 1,  activation='relu')(conv4)

#drop7 = Dropout(0.5)(dense7)

output = Dense(2048, activation='tanh')(dense7)
#output = Conv2D(2048, 1,  activation='relu')(conv5)

model_embed_visual_gabor = Model(inputs = feat, outputs = output)
model_embed_visual_gabor.summary()

#opt = Adagrad(learning_rate=0.0001, initial_accumulator_value=0.1, epsilon=1e-07)
opt = Adam(learning_rate = 0.0001, beta_1 = 0.9, beta_2 = 0.999)
#opt = SGD(learning_rate = 0.001, decay = 1e-6, momentum = 0.9, nesterov = True)
model_embed_visual_gabor.compile(optimizer = opt, loss = 'mse', metrics = ['accuracy'])



Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1, 1, 2048)]      0         
                                                                 
 dense (Dense)               (None, 1, 1, 1024)        2098176   
                                                                 
 dense_1 (Dense)             (None, 1, 1, 768)         787200    
                                                                 
 dense_2 (Dense)             (None, 1, 1, 512)         393728    
                                                                 
 dense_3 (Dense)             (None, 1, 1, 768)         393984    
                                                                 
 dense_4 (Dense)             (None, 1, 1, 1024)        787456    
                                                                 
 dense_5 (Dense)             (None, 1, 1, 2048)        2099

In [13]:
input_feat = train_visual_features
print(input_feat.shape)

output_feat = transformed_handcrafted_features
print(output_feat.shape)

X_train, X_val, y_train, y_val = train_test_split(input_feat, output_feat, test_size = 0.2, random_state = 42)


class FeatureGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

batch_size = 16
train_gen = FeatureGenerator(X_train, y_train, batch_size)   
val_gen = FeatureGenerator(X_val, y_val, batch_size)

(23520, 1, 1, 2048)
(23520, 1, 1, 2048)


In [14]:
print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)


train_summary = model_embed_visual_gabor.fit(train_gen, epochs = 200, verbose = 1, callbacks = None, 
                                      validation_data = val_gen, shuffle = True, 
                                      steps_per_epoch = len(X_train)//batch_size, 
                                      validation_steps = len(X_val)//batch_size)


(18816, 1, 1, 2048) (4704, 1, 1, 2048) (18816, 1, 1, 2048) (4704, 1, 1, 2048)
Epoch 1/200
1176/1176 [==============================] - 6s 5ms/step - loss: 0.0024 - accuracy: 0.2020 - val_loss: 0.0018 - val_accuracy: 0.2946
Epoch 2/200
1176/1176 [==============================] - 5s 5ms/step - loss: 0.0018 - accuracy: 0.2982 - val_loss: 0.0018 - val_accuracy: 0.3276
Epoch 3/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0018 - accuracy: 0.3241 - val_loss: 0.0018 - val_accuracy: 0.3365
Epoch 4/200
1176/1176 [==============================] - 5s 5ms/step - loss: 0.0018 - accuracy: 0.3361 - val_loss: 0.0017 - val_accuracy: 0.3518
Epoch 5/200
1176/1176 [==============================] - 5s 5ms/step - loss: 0.0017 - accuracy: 0.3397 - val_loss: 0.0017 - val_accuracy: 0.3416
Epoch 6/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0017 - accuracy: 0.3461 - val_loss: 0.0017 - val_accuracy: 0.3408
Epoch 7/200
1176/1176 [=============================

1176/1176 [==============================] - 5s 5ms/step - loss: 0.0012 - accuracy: 0.6350 - val_loss: 0.0018 - val_accuracy: 0.3397
Epoch 113/200
1176/1176 [==============================] - 5s 5ms/step - loss: 0.0012 - accuracy: 0.6324 - val_loss: 0.0018 - val_accuracy: 0.3289
Epoch 114/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0012 - accuracy: 0.6352 - val_loss: 0.0018 - val_accuracy: 0.3372
Epoch 115/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0012 - accuracy: 0.6404 - val_loss: 0.0018 - val_accuracy: 0.3372
Epoch 116/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0012 - accuracy: 0.6397 - val_loss: 0.0018 - val_accuracy: 0.3331
Epoch 117/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0012 - accuracy: 0.6446 - val_loss: 0.0018 - val_accuracy: 0.3293
Epoch 118/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0012 - accuracy: 0.6439 - val_loss: 0.0018 - val_ac

Epoch 168/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0011 - accuracy: 0.7066 - val_loss: 0.0019 - val_accuracy: 0.3242
Epoch 169/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0011 - accuracy: 0.7093 - val_loss: 0.0019 - val_accuracy: 0.3238
Epoch 170/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0011 - accuracy: 0.7087 - val_loss: 0.0019 - val_accuracy: 0.3280
Epoch 171/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0011 - accuracy: 0.7120 - val_loss: 0.0019 - val_accuracy: 0.3180
Epoch 172/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0011 - accuracy: 0.7117 - val_loss: 0.0019 - val_accuracy: 0.3297
Epoch 173/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0011 - accuracy: 0.7093 - val_loss: 0.0019 - val_accuracy: 0.3223
Epoch 174/200
1176/1176 [==============================] - 5s 4ms/step - loss: 0.0011 - accuracy: 0.7126 - val_loss: 0

In [15]:
name = 'model1_embed_visual_gabor_AWA2_200eph_16bch_adam_tanh_0.0001lr'
save_path = 'C:/Users/Admin/Sushree_Codes/Sush/Results/'

model_embed_visual_gabor.save_weights(save_path + 'Wt_' + name + '.h5')


# Results

input shape = 2048
output shape = 2048

1. model1: input -> 1024 -> 768 -> 512 -> 768 -> 1024 -> output
Input and Output are not normalized

1.1. opt = SGD, lr = 0.01, bch = 16, eph = 200
loss: 0.0018 - accuracy: 0.2997 - val_loss: 0.0017 - val_accuracy: 0.3095

1.2. opt = SGD, lr = 0.001, bch = 16, eph = 200
loss: 0.0019 - accuracy: 0.1174 - val_loss: 0.0018 - val_accuracy: 0.1182

1.3 opt = Adam, lr = 0.01, bch = 16, eph = 200
loss: 0.0018 - accuracy: 0.1917 - val_loss: 0.0018 - val_accuracy: 0.1966

1.4 opt = Adam, lr = 0.001, bch = 16, eph = 200
loss: 0.0018 - accuracy: 0.1917 - val_loss: 0.0018 - val_accuracy: 0.1966

1.5 opt = Adam, lr = 0.0001, bch = 16, eph = 200
loss: 0.0014 - accuracy: 0.8510 - val_loss: 0.0018 - val_accuracy: 0.3822

1.6 opt = Adam, lr = 0.00001, bch = 16, eph = 200
loss: 0.0016 - accuracy: 0.6742 - val_loss: 0.0017 - val_accuracy: 0.3525

1.7 opt = Adagrad, lr = 0.001, bch = 16, eph = 200
loss: 0.0019 - accuracy: 3.1888e-04 - val_loss: 0.0018 - val_accuracy: 0.0011



2. model1: input -> 1024 -> 768 -> 512 -> 768 -> 1024 -> output
Input and Output are normalized

2.1 opt = Adam, lr = 0.001, bch = 16, eph = 200
loss: 0.0018 - accuracy: 0.1036 - val_loss: 0.0019 - val_accuracy: 0.0910

2.2 opt = Adam, lr = 0.0001, bch = 16, eph = 200
loss: 0.0012 - accuracy: 0.4780 - val_loss: 0.0021 - val_accuracy: 0.0935

2.3 opt = Adam, lr = 0.00001, bch = 16, eph = 200
loss: 0.0280 - accuracy: 0.1599 - val_loss: 0.0281 - val_accuracy: 0.1050

2.4 opt = Adagrad, lr = 0.001, bch = 16, eph = 200
loss: 0.0767 - accuracy: 0.0022 - val_loss: 0.0767 - val_accuracy: 0.0011

2.5 opt = Adagrad, lr = 0.0001, bch = 16, eph = 200
loss: 0.0827 - accuracy: 0.0016 - val_loss: 0.0826 - val_accuracy: 0.0019



3. model1 + dropout: input -> 1024 -> D -> 768 -> D -> 512 -> D -> 768 -> D -> 1024 -> output
Input and Output are normalized

3.1 opt = Adam, lr = 0.0001, bch = 16, eph = 200, dropout = 0.2
loss: 0.0018 - accuracy: 0.1328 - val_loss: 0.0019 - val_accuracy: 0.1044

3.2 opt = Adam, lr = 0.0001, bch = 16, eph = 200, dropout = 0.1
loss: 0.0016 - accuracy: 0.1760 - val_loss: 0.0019 - val_accuracy: 0.1176

3.3 opt = Adam, lr = 0.0001, bch = 16, eph = 200, dropout = 0.5
loss: 0.0025 - accuracy: 0.0042 - val_loss: 0.0025 - val_accuracy: 0.0023

3.4 opt = Adagrad, lr = 0.001, bch = 16, eph = 200, dropout = 0.5
loss: 0.0168 - accuracy: 0.0013 - val_loss: 0.0165 - val_accuracy: 0.0068

3.5 opt = Adagrad, lr = 0.001, bch = 16, eph = 200, dropout = 0.2
loss: 0.0385 - accuracy: 0.0010 - val_loss: 0.0377 - val_accuracy: 0.0034

3.6 opt = Adagrad, lr = 0.001, bch = 16, eph = 200, dropout = 0.1
loss: 0.0492 - accuracy: 0.0015 - val_loss: 0.0484 - val_accuracy: 2.1259e-04



4. model2: input -> 1024 -> 896 -> 768 -> 512 -> 768 -> 896 -> 1024 -> output
Input and Output are not normalized

4.1 opt = Adam, lr = 0.0001, bch = 16, eph = 200
loss: 0.0013 - accuracy: 0.8637 - val_loss: 0.0018 - val_accuracy: 0.3865



5. model2 + dropout: input -> 1024 -> D -> 896 -> D -> 768 -> D -> 512 -> D -> 768 -> D -> 896 -> D -> 1024 -> output
Input and Output are not normalized

5.1 opt = Adam, lr = 0.0001, bch = 16, eph = 200, dropout = 0.1
loss: 0.0015 - accuracy: 0.6649 - val_loss: 0.0017 - val_accuracy: 0.3873

5.2 opt = Adam, lr = 0.0001, bch = 16, eph = 200, dropout = 0.2
loss: 0.0016 - accuracy: 0.5576 - val_loss: 0.0017 - val_accuracy: 0.3958

5.3 opt = Adam, lr = 0.0001, bch = 16, eph = 200, dropout = 0.5
loss: 0.0018 - accuracy: 0.1928 - val_loss: 0.0018 - val_accuracy: 0.2005


6. model3: input -> 1024 -> 512 -> 1024 -> output
Input and Output are not normalized

6.1 opt = Adam, lr = 0.0001, bch = 16, eph = 200
loss: 0.0015 - accuracy: 0.8023 - val_loss: 0.0017 - val_accuracy: 0.3903


7. modelconv1: input -> conv 1x1 1024 -> conv 1x1 768 -> conv 1x1 512 -> conv 1x1 768 -> conv 1x1 1024 -> output

7.1 opt = Adam, lr = 0.0001, bch = 16, eph = 200, inputs are not normalized
loss: 0.0014 - accuracy: 0.8554 - val_loss: 0.0018 - val_accuracy: 0.3716

7.2 opt = Adam, lr = 0.0001, bch = 16, eph = 200, inputs are normalized
loss: 0.0012 - accuracy: 0.4680 - val_loss: 0.0021 - val_accuracy: 0.0978

7.3 opt = Adam, act = tanh, lr = 0.0001, bch = 16, eph = 200, inputs are normalized (visual feat: 0 to 1, handcrafted: -1 to 1)
loss: 0.0062 - accuracy: 0.2375 - val_loss: 0.0084 - val_accuracy: 0.1167


With modified visual features
Visual features: extracted from last layer of pretrained ResNet101, no finetuning, size = 2048
Gabor features: features with 8 kernels are extracted, kernel pca with poly, size = 2048

8. modelconv1: input -> conv 1x1 1024 -> conv 1x1 768 -> conv 1x1 512 -> conv 1x1 768 -> conv 1x1 1024 -> output

8.1 modelconv1_embed_visual_gabor_AWA2_norm2feat_200eph_16bch_adam_tanh_0.0001lr 
opt = Adam, act = tanh, lr = 0.0001, bch = 16, eph = 200, inputs are normalized (visual feat: 0 to 1, handcrafted: -1 to 1)
loss: 0.0070 - accuracy: 0.1320 - val_loss: 0.0082 - val_accuracy: 0.0938

8.2 modelconv1_embed_visual_gabor_AWA2_200eph_16bch_adam_tanh_0.0001lr
opt = Adam, act = tanh, lr = 0.0001, bch = 16, eph = 200, inputs are not normalized
loss: 0.0011 - accuracy: 0.7359 - val_loss: 0.0019 - val_accuracy: 0.3185

8.3 modelconv1_embed_visual_gabor_AWA2_normfeat_200eph_16bch_adam_tanh_0.0001lr
opt = Adam, act = relu, lr = 0.0001, bch = 16, eph = 200, inputs are normalized (visual feat and handcrafted: 0 to 1)
loss: 0.0012 - accuracy: 0.4771 - val_loss: 0.0021 - val_accuracy: 0.0797


9. model1: input -> 1024 -> 768 -> 512 -> 768 -> 1024 -> output
Input and Output are not normalized

9.1 model1_embed_visual_gabor_AWA2_200eph_16bch_adam_relu_0.0001lr
opt = Adam, act = relu, lr = 0.0001, bch = 16, eph = 200
loss: 0.0014 - accuracy: 0.8332 - val_loss: 0.0018 - val_accuracy: 0.3335

9.2 model1_embed_visual_gabor_AWA2_200eph_16bch_adam_tanh_0.0001lr
opt = Adam, act = tanh, lr = 0.0001, bch = 16, eph = 200
loss: 0.0011 - accuracy: 0.7308 - val_loss: 0.0019 - val_accuracy: 0.3153

